In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sentence_transformers import SentenceTransformer

C:\Users\User\anaconda3\envs\claim-fact-check\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('taylor_swift_multiple_news_OR.csv')
df.head()

,url,title,description,maintext,date_publish
0,https://www.cnn.com/2023/04/09/entertainment/t...,Taylor Swift and Joe Alwyn break up after six ...,Taylor Swift and actor Joe Alwyn have broken u...,CNN —\nTaylor Swift and actor Joe Alwyn have b...,2023-04-09 17:19:35
1,https://www.cnn.com/2023/04/10/entertainment/t...,Did Taylor Swift drop a hint about her breakup...,Taylor Swift is known for dropping Easter eggs...,CNN —\nTaylor Swift is known for dropping East...,2023-04-10 13:43:11
2,https://www.nytimes.com/2023/04/10/style/taylo...,Taylor Swift Fans Grapple With Breakup Reports,After “Entertainment Tonight” and People publi...,"To quote Taylor Swift’s own lyrics, “The rumor...",2023-04-10 20:17:48
3,https://www.bbc.com/news/newsbeat-67588850,Taylor Swift: Publicist denies star secretly m...,Tree Paine has been part of the singer's team ...,Taylor Swift: Publicist denies star secretly m...,2023-12-01 11:14:18
4,https://www.nytimes.com/interactive/2023/08/11...,Taylor Swift’s Viral Era,"Fan demand broke Ticketmaster, and that was ju...","Fan demand broke Ticketmaster, and that was ju...",2023-08-11 15:41:36


In [9]:
df = df.dropna()

In [10]:
sentences = df.maintext.apply(lambda x: x.split('\n')) # df.maintext.apply(split_into_sentences)
sentences = [sentence for lst in sentences for sentence in lst]

In [14]:
numbers_of_sentences = df.maintext.apply(lambda x: x.split('\n')).apply(len)

sentence_to_id = []
for i, num in enumerate(numbers_of_sentences):
    sentence_to_id += [i] * num
    
assert len(sentences)==len(sentence_to_id)

In [15]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [16]:
embeddings = model.encode(sentences)

In [17]:
nn = NearestNeighbors(n_neighbors=3)
nn.fit(embeddings)

NearestNeighbors(n_neighbors=3)

In [18]:
query = "Did Taylor Swift recently break up with Joe Alywn?"

In [19]:
target = model.encode(query)

In [20]:
indices = nn.kneighbors(target[np.newaxis, :], return_distance=False)

In [21]:
relevant = np.array(sentences)[indices]
relevant = '. '.join(relevant[0])

In [22]:
prompt = f"""Based only on the context: {relevant}
Tell me if this statement is true/false: {query}
Answer in this format: <evidences in simple words> <conclude true/false>"""

In [23]:
print(prompt)

Based only on the context: Taylor Swift and actor Joe Alwyn have broken up after six years together.. A source close to Taylor confirmed the news to CNN saying “Taylor and Joe broke up a few weeks ago. They simply grew apart and plan to remain friends.”. Taylor Swift's publicist has hit out at rumours that the popstar secretly married ex-partner Joe Alwyn.
Tell me if this statement is true/false: Did Taylor Swift recently break up with Joe Alywn?
Answer in this format: <evidences in simple words> <conclude true/false>


In [27]:
print("True. The evidence provided states that Taylor Swift and Joe Alwyn have indeed broken up. A source close to Taylor confirmed the news to CNN, mentioning that they broke up a few weeks ago. Additionally, Taylor Swift's publicist addressed rumors about her secretly marrying Joe Alwyn, which implies that they are no longer together. Therefore, based on the evidence, it is true that Taylor Swift recently broke up with Joe Alwyn.")

True. The evidence provided states that Taylor Swift and Joe Alwyn have indeed broken up. A source close to Taylor confirmed the news to CNN, mentioning that they broke up a few weeks ago. Additionally, Taylor Swift's publicist addressed rumors about her secretly marrying Joe Alwyn, which implies that they are no longer together. Therefore, based on the evidence, it is true that Taylor Swift recently broke up with Joe Alwyn.
